# Validate Accuracy Scores

In [1]:
import os
import pandas as pd
from sklearn.metrics import root_mean_squared_error, r2_score
from scipy import stats
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
inputDict = {
    'JASMIN-NL':'/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_nl_age7-11_nat',
    'JASMIN-VL':'/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_vl_age7-11_nat',
    'SERDA-NL':'/vol/tensusers2/wharmsen/SERDA-fluency-features/comp1'
}

corrDF_list = []
combiDF_list = []
for corpus, basepath in inputDict.items():
    print(corpus)

    manFile = os.path.join(basepath,'06_manual_fluency_features/json-fluency-features/accuracy.tsv')
    manDF = pd.read_csv(manFile, sep = '\t', index_col=0).sort_index()

    asrSystem = 'whispert'
    autoFile = os.path.join(basepath,'05_automatic_fluency_features/' + asrSystem + '/accuracy.tsv')
    autoDF = pd.read_csv(autoFile, sep = '\t', index_col=0).sort_index()
    autoDF.columns = [asrSystem + '_' + x for x in autoDF.columns]

    accVars = list(manDF.columns)
    accVars.remove('other') if 'other' in list(manDF.columns) else list(manDF.columns)
    print('accVars: ', accVars)

    combiDF = pd.concat([autoDF, manDF], axis=1)
    # combiDF = combiDF.dropna()
    print('Length combiDF:', len(combiDF))
    combiDF_list.append(combiDF)

    outputList = []
    for accVar in accVars:
        # Select only the automatic and manual variable
        autoVar = asrSystem + '_' + accVar
        manVar = accVar
        varDF = combiDF.loc[:, [autoVar, manVar]].replace(0, np.nan, inplace=False).dropna()
        length = len(varDF)
        corr = stats.pearsonr(varDF[asrSystem + '_' + accVar], varDF[accVar])[0]
        outputList.append([accVar, length, corr])

    corrDF = pd.DataFrame(outputList, columns = ['var', 'N', corpus]).set_index('var')
    corrDF_list.append(corrDF)

outputCorrDF = pd.concat(corrDF_list, axis=1)

In [ ]:
outputCorrDF